In [ ]:
## atmorep_results 

In [ ]:
import zarr
import os
import xarray as xr
import numpy as np

In [ ]:
model_id = "idq6fyeanc"#"id2urb4mvu"#"idct1quho0"

In [ ]:
model_id_folder_path = f"/p/project1/deepacf/atmo-rep/patnala1/atmorep/results/{model_id}"
ensemble_path = f"results_{model_id}_epoch00000_ens.zarr"
pred_path = f"results_{model_id}_epoch00000_pred.zarr"
source_path = f"results_{model_id}_epoch00000_source.zarr"
target_path = f"results_{model_id}_epoch00000_target.zarr"

In [ ]:
target_store = zarr.ZipStore(os.path.join(model_id_folder_path, target_path))
target_group = zarr.group(store=target_store)

In [ ]:
pred_store = zarr.ZipStore(os.path.join(model_id_folder_path,pred_path))
pred_group = zarr.group(store=pred_store)

In [ ]:
field ='total_precip'
sample = '00012'

In [ ]:
list(pred_group.group_keys())

In [ ]:
list(target_group[field][f'sample={sample}'].keys())

In [ ]:
target_data = target_group[field][f'sample={sample}']['data'][0][:]
target_lats = target_group[field][f'sample={sample}']['lat'][:]
target_lons = target_group[field][f'sample={sample}']['lon'][:]
target_datetime = target_group[field][f'sample={sample}']['datetime'][:]

In [ ]:
pred_data = pred_group[field][f'sample={sample}']['data'][0][:]
pred_lats = pred_group[field][f'sample={sample}']['lat'][:]
pred_lons = pred_group[field][f'sample={sample}']['lon'][:]
pred_datetime = pred_group[field][f'sample={sample}']['datetime'][:]

In [ ]:
def math_to_geolats(lats_math):
    """
    Convert 'mathematical' latitudes as used in AtmoRep to geographical latitudes.
    Example conversions:
        Mathematical latitude: 0° -> Geographical latitude: 90°N
        Mathematical latitude: 180° -> Geographical latitude: -90°N/90°S
    :param lats_math: 1D-array with mathematical latitudes
    :return lats_geo: 1D-array with geographical latitudes
    """
    lats_geo = np.where(lats_math > 90, -lats_math + 90, np.abs(lats_math - 90.))

    return lats_geo

def geo_to_mathlats(lats_geo):
    """
    Convert geographical latitude to 'mathematical' latitudes as used in AtmoRep.
    Example conversions:
        Geographical latitude: 90°N ->  Mathematical latitude: 0°N
        Geographical latitude: -90°N -> Mathematical latitude: 180°
    :param lats_math: 1D-array with geographical latitudes
    :return lats_geo: 1D-array with mathematical latitudes
    """
    lats_math = np.where(lats_geo > 0, np.abs(lats_geo - 90), -lats_geo + 90.)
    return lats_math

def get_latlon_arr(lat_range, lon_range, res):
    lats = np.arange(lat_range[0], lat_range[1] + res/10., res)
    lons = np.arange(lon_range[0], lon_range[1] + res/10., res)

    return lats, lons

In [ ]:
target_array = xr.DataArray( target_data, dims=["time","lat","lon"],
                            coords = {"time": target_datetime, "lat":math_to_geolats(target_lats), "lon":np.where(target_lons > 180.0, target_lons - 360.0, target_lons)})

pred_array = xr.DataArray( pred_data, dims=["time","lat","lon"],
                            coords = {"time": pred_datetime, "lat":math_to_geolats(pred_lats), "lon":np.where(pred_lons > 180.0, pred_lons - 360.0, pred_lons)})


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

tidx = 4
#val_range = {"vmin": 0, "vmax": 2.}


In [ ]:
target_array.isel({"time": tidx}).plot()#**val_range)
plt.show()

In [ ]:
pred_array.isel({"time": tidx}).plot()#**val_range)
plt.show()

In [ ]:
np.mean(pred_data - target_data)